## Intro to Tracktable - Fall 2023
Documentation Link: *https://tracktable.readthedocs.io/en/latest/*

We just spent the last couple notebooks manipulating flight data and while we were mostly successful it took a while to build up out database and manipulate our flight trajectories. The library you'll use in this notebook (Tracktable) was created and maintained by some of the lovely folks at Sandia to analyze geospatial data.

In the following cells replace '❌' with their corsponding values for our dataset and procedures.

In [1]:
#Import needed libraries
import os.path
import tracktable

from tracktable.core import geomath
from tracktable.domain.terrestrial import TrajectoryPointReader
from tracktable.applications.assemble_trajectories import AssembleTrajectoryFromPoints
from tracktable.render.render_trajectories import render_trajectories

from datetime import datetime, timedelta

tracktable.__version__

'1.6.0'

Let's take a look at the file we're using, it should contain flight information.

In [2]:
%%bash
head "/anvil/projects/tdm/corporate/sandia-trajectory/previous_files/flight/data/raw_data/asdi_2014_07_01_h121314_safe.tsv"

OBJECT_ID	UPDATE_TIME	LONGITUDE	LATITUDE	SPEED	HEADING	ALTITUDE
N404TC	2014-07-01 12:00:00	-67.05	45.6833	430	56	39000
N617BG	2014-07-01 12:00:00	-97.7833	30.1161	156	246	5500
EJA644	2014-07-01 12:00:00					
N748W	2014-07-01 12:00:00	-92	40.1611	451	6	38500
N802UP	2014-07-01 12:00:00					
SWA4414	2014-07-01 12:00:00	-89.4131	37.8119	457	115	33000
N536M	2014-07-01 12:00:00					
N2095J	2014-07-01 12:00:00					7000
N81TE	2014-07-01 12:00:00					


The reader tells Tracktable where our data is stored and what information exists in each column.

In [3]:
#Read in the file, let it know what the comment & delimiter character is
data_filename = os.path.join("/anvil/projects/tdm/corporate/sandia-trajectory/previous_files/flight/data/raw_data/asdi_2014_07_01_h121314_safe.tsv")
inFile = open(data_filename, 'r')
reader = TrajectoryPointReader()
reader.input = inFile
reader.comment_character = '#' 	#What character is used for comments
reader.field_delimiter = '\t' 	#What character "breaks" each data value ex: Comma-Separated Values

#Columns start at 0, ex: 0 is column A, 2 is column C
reader.object_id_column = 0 	#What column holds the object ID
reader.timestamp_column = 1 	#What column holds the timestamp
reader.coordinates[1] = 3		#What column holds LAT data
reader.coordinates[0] = 2		#What column holds LONG data
reader.set_real_field_column('speed', 4) #Extra data (heading)
reader.set_real_field_column('heading', 5) #Extra data (heading)
reader.set_real_field_column('altitude', 6) #Extra data (altitude)

Nothing to change in the next cell, just a way to check our work.

In [4]:
#Test to see if data has been imported correctly.
limit = 5					# Used to limit how many results we see
for i, x in enumerate(reader):
    if i >= limit: break	# Exits a loop early
    print(x)				# Print a line from reader

[N404TC@ 2014-07-01 12:00:00: (-67.05, 45.6833) Properties: ( {altitude [real]: 39000}, {heading [real]: 56}, {speed [real]: 430})]
[N617BG@ 2014-07-01 12:00:00: (-97.7833, 30.1161) Properties: ( {altitude [real]: 5500}, {heading [real]: 246}, {speed [real]: 156})]
[N748W@ 2014-07-01 12:00:00: (-92, 40.1611) Properties: ( {altitude [real]: 38500}, {heading [real]: 6}, {speed [real]: 451})]
[SWA4414@ 2014-07-01 12:00:00: (-89.4131, 37.8119) Properties: ( {altitude [real]: 33000}, {heading [real]: 115}, {speed [real]: 457})]
[AAL40@ 2014-07-01 12:00:00: (-85.7925, 41.8717) Properties: ( {altitude [real]: 27000}, {heading [real]: 107}, {speed [real]: 480})]


The builder stitches together rows with the same object_id then will split it into flights based on temporal gaps or distance. There are default thresholds but let's update it to what we have used before.
 * minium length of 10
 * temporal gap limit of 30 minutes

In [5]:
#Combine datapoints together using the object_id
builder = AssembleTrajectoryFromPoints()
builder.input = reader
builder.minimum_length = 10
builder.separation_time = timedelta(minutes = 30)
traj = list(builder.trajectories())
print(len(traj), '〖10815〗flights built! ✈')

print(f'The type of traj is {type(traj)}')
print(f'traj is a list of {type(traj[0])}')

INFO:tracktable.applications.assemble_trajectoriesAssembleTrajectoryFromPoints:New trajectories will be declared after a separation of None distance units between two points or a time lapse of at least 0:30:00 (hours, minutes, seconds).
INFO:tracktable.applications.assemble_trajectoriesAssembleTrajectoryFromPoints:Trajectories with fewer than 10 points will be discarded.


[2023-10-06 11:47:02.262779] [0x00007f0d315aa740] [info]    Done reading points. Generated 778983 points correctly and discarded 29775 due to parse errors.



INFO:tracktable.applications.assemble_trajectoriesAssembleTrajectoryFromPoints:Done assembling trajectories. 10815 trajectories produced and 1355 discarded for having fewer than 10 points.


10815 〖10815〗flights built! ✈
The type of traj is <class 'list'>
traj is a list of <class 'tracktable.lib._terrestrial.TrajectoryTerrestrial'>


Unpacking our first flight in traj we can see all the points included. They should be in time sequential order and share the same object_id, which is what we would expect from a trajectory.

In [6]:
print(*traj[0])

[N567LT@ 2014-07-01 12:00:37: (-78.1106, 35.9125) Properties: ( {altitude [real]: 6000}, {heading [real]: 211}, {speed [real]: 180})] [N567LT@ 2014-07-01 12:01:26: (-78.1325, 35.885) Properties: ( {altitude [real]: 6000}, {heading [real]: 213}, {speed [real]: 189})] [N567LT@ 2014-07-01 12:01:38: (-78.145, 35.8681) Properties: ( {altitude [real]: 6000}, {heading [real]: 211}, {speed [real]: 180})] [N567LT@ 2014-07-01 12:02:37: (-78.1769, 35.8278) Properties: ( {altitude [real]: 6000}, {heading [real]: 205}, {speed [real]: 182})] [N567LT@ 2014-07-01 12:03:38: (-78.2114, 35.7853) Properties: ( {altitude [real]: 6000}, {heading [real]: 208}, {speed [real]: 177})] [N567LT@ 2014-07-01 12:04:39: (-78.2436, 35.7492) Properties: ( {altitude [real]: 5200}, {heading [real]: 215}, {speed [real]: 166})] [N567LT@ 2014-07-01 12:05:25: (-78.2656, 35.7214) Properties: ( {altitude [real]: 4500}, {heading [real]: 213}, {speed [real]: 183})] [N567LT@ 2014-07-01 12:05:40: (-78.2778, 35.7044) Properties: ( 

A single flight is way more than just a list of points, its actually a class so it store more information.

In [8]:
print(f'Object_id: {traj[0][0].object_id}')                     # Trajectory 0 in list, point 0 in trajectory
print(f'Latitude: {traj[0][0][1]}')                        # Trajectory 0 in list, point 0 in trajectory, coordinate[1]
print(f"Heading: {traj[0][0].properties['heading']}")      
print(f'Data Points: {len(traj[0])}')      

Object_id: N567LT
Latitude: 35.9125
Heading: 211.0
Data Points: 10


Now (with the power of tracktable) flight features that may have taken multiple lines to obtain can be computed with ease.

Let's compute the flight's length and convex hull ratio using one line of python for each.
*https://tracktable.readthedocs.io/en/latest/api/python/tracktable.core.geomath.html*

In [21]:
flightOfInterest = traj[3000]

####################         YOUR CODE         ####################

flightLength = tracktable.core.geomath.length(flightOfInterest)

convexHullRatio = tracktable.core.geomath.convex_hull_aspect_ratio(flightOfInterest)

####################         END CODE          ####################

print(f'Flight {flightOfInterest[0].object_id} has a length of {flightLength:.2f}〖1566.34〗km.')
print(f'and a convex hull ratio of {convexHullRatio:.2f}〖24.33〗km^2 / km.')

Flight AWE2064 has a length of 1566.34〖1566.34〗km.
and a convex hull ratio of 24.33〖24.33〗km^2 / km.


As a bouns it comes with a great way to view flights too!

In [21]:
#Render the first flight in the list
render_trajectories(flightOfInterest)

We can actually render multiple flights at once! Render four flights from the center of our dataset.

In [13]:
####################         YOUR CODE         ####################

centerFour = [traj[0],traj[1],traj[2],traj[3]]

####################         END CODE          ####################
render_trajectories(centerFour)

Determine which flight has the most datapoints in our dataset.

In [24]:
####################         YOUR CODE         ####################
largest = max(traj, key = len)

####################         END CODE          ####################        
        
print(f'The largest flight has {len(largest)}〖292〗datapoints!')
render_trajectories(largest)

The largest flight has 292〖292〗datapoints!


In [21]:
len(traj[0])

10

Determine which flight is the longest in our dataset.

In [25]:
####################         YOUR CODE         ####################

longest = max(traj, key = tracktable.core.geomath.length)

####################         END CODE          ####################

print(f'The longest flight is {geomath.length(longest):.2f}〖6767.54〗km!')
render_trajectories(longest)

The longest flight is 6767.54〖6767.54〗km!
